# Dataset Generation - Part 2

Now that we have our join combinations, we need to use it with some extra information to generate our dialogues. We will also use LLMs to generate our dialogue dataset, and use human expert to check if dialogues make sense and if the sql query generated as label is also correct.

Our prompt to llm will be something like this:
```python
"""
Hey friend, take this joins combinations: {join_combinations}
Consider that in this joins, we involve this tables: {tables_involved}

Here are some informations about this tables: 
<big piece of context that have create table clauses and columns values examples>

<Instructions>
<Example of dialogue in the output format>
"""
```

It works very nice because isn't hard to llms create valid SQL queries when they know the structure of the table (we give it in create table clause) and column values so llm don't hallucinate about what that can be.

We have already created in last notebook the join combinations, and it is on our csv, but we need to:
1. Extract create table clauses for each table in schema.
2. Get from database example values available for each column.

So let's do it!

## Extracting table DDLs:

In [4]:
import re
from dotenv import load_dotenv

load_dotenv()

with open("mondial/schema.txt", "r") as f:
    schema = f.read()

def extract_ddls_from_schema(schema_text):

    pattern = r'-- auto-generated definition\s+create table (MONDIAL_\w+)\s*\(\s*([\s\S]+?)\)\s*/'
    matches = re.findall(pattern, schema_text)
    
    ddl_dict = {}
    for table_name, ddl_content in matches:
        full_ddl = f"CREATE TABLE {table_name} (\n{ddl_content}\n)"
        ddl_dict[table_name] = full_ddl
    
    return ddl_dict

table_ddls = extract_ddls_from_schema(schema)

print(f"Total of tables found: {len(table_ddls)}")
print("\nExample of DDL for MONDIAL_COUNTRY:")
print(table_ddls.get("MONDIAL_COUNTRY", "Table not found"))

Total of tables found: 40

Example of DDL for MONDIAL_COUNTRY:
CREATE TABLE MONDIAL_COUNTRY (
NAME        VARCHAR2(50),
    CODE        VARCHAR2(4),
    CAPITAL     VARCHAR2(50),
    PROVINCE    VARCHAR2(50),
    AREA        NUMBER,
    POPULATION  NUMBER,
    META_REPCOL VARCHAR2(4000)

)


## Getting column values examples

### !! In my case, my company host MONDIAL on Oracle, so i'll do the stuff in next cell, you may change it to adapt to your database !!

In [6]:
import json
import oracledb

# 
with open("mondial/mondial_db_connection.json", "r") as file:
    db_config = json.load(file)

db_host = db_config["DB_HOST"]
db_port = db_config["DB_PORT"]
db_user = db_config["DB_USER_NAME"]
db_pass = db_config["DB_PASS"]
service_name = db_config["SERVICE_NAME"]

dsn = f"{db_host}:{db_port}/{service_name}"
connection = oracledb.connect(
    user=db_user,
    password=db_pass,
    dsn=dsn
)
cursor = connection.cursor()

# Get all tables names
cursor.execute("SELECT TABLE_NAME FROM USER_TABLES WHERE TABLE_NAME LIKE 'MONDIAL_%'")
tables = [row[0] for row in cursor.fetchall()]

table_examples = {}  # Dictionary to store examples by table

# For each table, search column names and examples of values
for table in tables:
    # Query to get column names
    cursor.execute(f"SELECT COLUMN_NAME FROM USER_TAB_COLUMNS WHERE TABLE_NAME = '{table}'")
    columns = [row[0] for row in cursor.fetchall()]
    
    columns_examples = {}
    for column in columns:
        # Get up to 20 distinct values
        try:
            cursor.execute(f"SELECT DISTINCT {column} FROM {table} WHERE {column} IS NOT NULL FETCH FIRST 20 ROWS ONLY")
            examples = []
            for row in cursor.fetchall():
                value = row[0]
                # If the value is a LOB, read the content
                if isinstance(value, oracledb.LOB):
                    value = value.read()
                examples.append(value)
            
            columns_examples[column] = examples
        except Exception as e:
            columns_examples[column] = ["<error to get examples>"]

    table_examples[table] = columns_examples

cursor.close()
connection.close()

print(f"Examples obtained for {len(table_examples)} tables")
print("\nExample for MONDIAL_COUNTRY:")
print(json.dumps(table_examples.get("MONDIAL_COUNTRY", {}), indent=2, default=str)[:500] + "...")

Examples obtained for 59 tables

Example for MONDIAL_COUNTRY:
{
  "NAME": [
    "Austria",
    "Lithuania",
    "Poland",
    "Faroe Islands",
    "Finland",
    "Ireland",
    "Malta",
    "Georgia",
    "Bangladesh",
    "Myanmar",
    "Thailand",
    "Israel",
    "Indonesia",
    "Papua New Guinea",
    "Saudi Arabia",
    "Syria",
    "Lebanon",
    "Anguilla",
    "United States",
    "Haiti"
  ],
  "CODE": [
    "A",
    "AFG",
    "AG",
    "AL",
    "AMSA",
    "AND",
    "ANG",
    "ARM",
    "ARU",
    "AUS",
    "AXA",
    "AZ",
    "B",
    "B...


## Now we have the create tables, example of values, lets load the joins and get into prompting

In [7]:
import pandas as pd

# Load the file with the optimal join combinations
join_combinations_df = pd.read_csv("mondial/optimal_join_combinations.csv")

# Create a list of objects for each join combination
combination_data = []
for idx, row in join_combinations_df.iterrows():
    tipo = row['Type']
    combinacao = row['Combination']
    tabelas_str = row['Tables']
    condicoes_join = row['Join Conditions']
    
    # Extract list of tables from string
    tabelas_list = [t.strip() for t in tabelas_str.strip('"').split(',')]
    
    combination_data.append({
        "combination_id": f"{tipo}_{combinacao}",
        "tables": tabelas_list,
        "combination_str": condicoes_join
    })

print(f"Total of join combinations loaded: {len(combination_data)}")
print("\nExample of combination:")
print(json.dumps(combination_data[0], indent=2))

Total of join combinations loaded: 50

Example of combination:
{
  "combination_id": "2_1",
  "tables": [
    "MONDIAL_GEO_MOUNTAIN",
    "MONDIAL_MOUNTAIN",
    "MONDIAL_MOUNTAINONISLAND"
  ],
  "combination_str": "MONDIAL_GEO_MOUNTAIN.['MONDIAL_GEO_MOUNTAIN.MOUNTAIN=MONDIAL_MOUNTAIN.NAME']; MONDIAL_MOUNTAINONISLAND.['MONDIAL_MOUNTAINONISLAND.MOUNTAIN=MONDIAL_MOUNTAIN.NAME']"
}


## As we did in last notebook, for best pratices we will setup a output format to the dialogue, since it need to be well formatted to get into json.

In [8]:
from llm_config import LLMConfig
from pydantic import BaseModel, Field
from typing import List

dialogue_generator = LLMConfig(provider="azure").get_llm(model="o3-mini", reasoning_effort="high")

class GroundTruth(BaseModel):
    tables_from_schema_linking: List[str] = Field(..., description="List of tables involved in the query.")
    golden_sql: str = Field(..., description="Valid SQL query corresponding to the question.")

class Interaction(BaseModel):
    interaction_id: str = Field(..., description="ID of the interaction.")
    speaker: str = Field(..., description="Who is speaking (always 'User').")
    utterance: str = Field(..., description="The natural language question.")
    intention: str = Field(..., description="Is the real intention of the user, it must be a natural language version of the question, considering that you don't now the global context of the dialogue or database information. Basically a compact version of the question, that agent must infer from the global context.")
    ground_truths: GroundTruth = Field(..., description="Details of the ground truth for the query.")

class Experiment(BaseModel):
    experiment_id: str = Field(..., description="Unique ID of the experiment.")
    total_expected_interactions: int = Field(..., description="Total number of interactions in the dialogue.")
    interactions: List[Interaction] = Field(..., description="List of interactions in the dialogue.")

dialogue_generator_with_structured_output = dialogue_generator.with_structured_output(Experiment)

# Now for each join combination, we will use all information we collected to create a personalized prompt.

In [10]:
# Prompts for each join combination
prompts = []
for idx, comb_obj in enumerate(combination_data):
    experiment_id = str(idx + 1)  # IDs start from 1
    join_str = comb_obj["combination_str"]
    tables_involved = comb_obj["tables"]
    joins_len = len(set(tables_involved))

    # For each involved table, we search the DDL and examples
    table_contexts = ""
    for table in tables_involved:
        ddl_text = table_ddls.get(table, "DDL not found.")
        examples = table_examples.get(table, {})
        
        # Convert datetime objects to string before serializing to JSON
        processed_examples = {}
        for key, value in examples.items():
            if isinstance(value, list):
                processed_examples[key] = [str(item) if hasattr(item, 'isoformat') else item for item in value]
            else:
                processed_examples[key] = str(value) if hasattr(value, 'isoformat') else value
        
        examples_str = json.dumps(processed_examples, indent=2, ensure_ascii=False)
        table_contexts += f"\nTabela: {table}\nDDL:\n{ddl_text}\n\nValores de Exemplo:\n{examples_str}\n\n"

    prompt = f"""
You are an advanced dialog generation agent. Your task is to generate a dialog experiment consisting of exactly {joins_len} interactions based on the database context provided below.

#Join Combination Context:
{join_str}

#Tables Involved:
{', '.join(tables_involved)}

#Table Details:
{table_contexts}

Generate an experiment following the JSON structure below. The experiment must include exactly 3 user interactions, each with:
- "interaction_id": a string identifier.
- "speaker": always "User".
- "utterance": the user's question expressed naturally.
- "intention": an exact or contextually refined rewrite of the "utterance" that reflects what the user really intends. If no additional context is needed, "intention" should be identical to "utterance". For follow-up queries, integrate the previous context to form a refined natural language version.
- "ground_truths": an object containing:
  - "tables_from_schema_linking": a list of the tables involved.
  - "danke_sql": a valid SQL query that can be executed on the provided schema. The SQL must be accurate and specific, and if sample values are used, they should serve only as filters—not to overly restrict or specify the general question.

Expected JSON format, THAT'S AN EXAMPLE, YOU MUST FOLLOW THIS STRUCTURE, NOTHING ELSE:
{{
  "experiment_id": "{experiment_id}",
  "total_expected_interactions": 3,
  "interactions": [
    {{
      "interaction_id": "1",
      "speaker": "User",
      "utterance": "What are the biggest city in the world?",
      "intention": "What are the biggest city in the world?",
      "ground_truths": {{
         "tables_from_schema_linking": [ ... ],
         "danke_sql": "<Valid SQL query corresponding to the question>"
      }}
    }},
    {{
      "interaction_id": "2",
      "speaker": "User",
      "utterance": "What are the area of this city?",
      "intention": "What are the area of <city retrieved in the previous interaction>?",
      "ground_truths": {{
         "tables_from_schema_linking": [ ... ],
         "danke_sql": "<Valid SQL query for the follow-up question>"
      }}
    }},
    {{
      "interaction_id": "3",
      "speaker": "User",
      "utterance": "And tell me also in wich continent it is?",
      "intention": "In wich continent is <city retrieved in the previous interaction>?",
      "ground_truths": {{
         "tables_from_schema_linking": [ ... ],
         "danke_sql": "<Valid SQL query for the additional question>"
      }}
    }}
  ]
}}

As you see, follow up questions are always related to the previous one, and the intention is a natural language version of the question, considering that you don't now the global context of the dialogue.

You must pay extra attention to the "intention" field, it must be a natural language version of the question, considering that you don't now the global context of the dialogue or database information, so don't use database information in the intention.

Try to generate a dialogue that the answer is not a giant table scan, more specific the better.
"""

    prompts.append((experiment_id, prompt.strip()))

print(f"Total de prompts criados: {len(prompts)}")
print("\nExemplo de prompt:")
print(prompts[0][1] + "...")

Total de prompts criados: 50

Exemplo de prompt:
You are an advanced dialog generation agent. Your task is to generate a dialog experiment consisting of exactly 3 interactions based on the database context provided below.

#Join Combination Context:
MONDIAL_GEO_MOUNTAIN.['MONDIAL_GEO_MOUNTAIN.MOUNTAIN=MONDIAL_MOUNTAIN.NAME']; MONDIAL_MOUNTAINONISLAND.['MONDIAL_MOUNTAINONISLAND.MOUNTAIN=MONDIAL_MOUNTAIN.NAME']

#Tables Involved:
MONDIAL_GEO_MOUNTAIN, MONDIAL_MOUNTAIN, MONDIAL_MOUNTAINONISLAND

#Table Details:

Tabela: MONDIAL_GEO_MOUNTAIN
DDL:
CREATE TABLE MONDIAL_GEO_MOUNTAIN (
MOUNTAIN    VARCHAR2(50),
    COUNTRY     VARCHAR2(4),
    PROVINCE    VARCHAR2(50),
    META_REPCOL VARCHAR2(4000)

)

Valores de Exemplo:
{
  "MOUNTAIN": [
    "Chappal Waddi",
    "Tamgak",
    "Huascaran",
    "Licancabur",
    "Cerro Tristeza",
    "Hiru Erregeen Mahaia",
    "Moncayo",
    "Jabal Sawda",
    "Süphan Dagi",
    "Botew",
    "Kanlaon",
    "Chimborazo",
    "Wolf",
    "Pico Mogotón",
    "P

## I really recommend you to read the example prompt that I've printed, to understand all the information we are prompting to LLM. Now for each prompt created, let's send to llm and generate the dialogues.

In [ ]:
import os

output_file = "mondial/dialogue_dataset.json"

# Check if the file already exists and load existing data
dataset = []
last_processed_index = -1
if os.path.exists(output_file):
    try:
        with open(output_file, "r", encoding="utf-8") as f:
            existing_data = json.load(f)
            dataset = existing_data.get("dataset", [])
            
            # Find the last processed experiment_id (good for continue the process in case of interruption)
            if dataset:
                processed_ids = [exp.get("experiment_id") for exp in dataset if "experiment_id" in exp]
                for i, (exp_id, _) in enumerate(prompts):
                    if exp_id in processed_ids:
                        last_processed_index = i
                
                print(f"Found {len(dataset)} already processed experiments. Continuing from index {last_processed_index + 1}")
    except Exception as e:
        print(f"Error to load existing file: {e}")
        print("Starting with a new dataset")

# Process the remaining prompts
for i, (experiment_id, prompt) in enumerate(prompts):
    # Skip already processed
    if i <= last_processed_index:
        continue
    
    print(f"Processing experiment {i+1}/{len(prompts)} (ID: {experiment_id})")
    
    try:
        # Call to LLM with structured output based on Experiment model
        experiment_output = dialogue_generator_with_structured_output.invoke(prompt)
        experiment_dict = experiment_output.dict()

        # If necessary, adjust the key name for the tables
        for interaction in experiment_dict.get("interactions", []):
            if "ground_truths" in interaction:
                gt = interaction["ground_truths"]
                if "tables_envolved" in gt:
                    gt["tables_from_schema_linking"] = gt.pop("tables_envolved")

        dataset.append(experiment_dict)
        print(f"Experiment generated: {i} (ID: {experiment_id})")
        
        # Salvar após cada experimento para evitar perda de dados
        final_dataset = {"dataset": dataset}
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(final_dataset, f, ensure_ascii=False, indent=2)
        
        print(f"Dataset updated and saved in {output_file}")
    
    except Exception as e:
        print(f"Error to process experiment {experiment_id}: {e}")

print(f"Completed. Total of {len(dataset)} experiments in the dataset.")

Processing experiment 1/50 (ID: 1)
Experiment generated: 0 (ID: 1)
Dataset updated and saved in mondial/dialogue_dataset.json
Processing experiment 2/50 (ID: 2)
Experiment generated: 1 (ID: 2)
Dataset updated and saved in mondial/dialogue_dataset.json
Processing experiment 3/50 (ID: 3)
Experiment generated: 2 (ID: 3)
Dataset updated and saved in mondial/dialogue_dataset.json
Processing experiment 4/50 (ID: 4)
Experiment generated: 3 (ID: 4)
Dataset updated and saved in mondial/dialogue_dataset.json
Processing experiment 5/50 (ID: 5)
Experiment generated: 4 (ID: 5)
Dataset updated and saved in mondial/dialogue_dataset.json
Processing experiment 6/50 (ID: 6)
Experiment generated: 5 (ID: 6)
Dataset updated and saved in mondial/dialogue_dataset.json
Processing experiment 7/50 (ID: 7)
Experiment generated: 6 (ID: 7)
Dataset updated and saved in mondial/dialogue_dataset.json
Processing experiment 8/50 (ID: 8)
Experiment generated: 7 (ID: 8)
Dataset updated and saved in mondial/dialogue_datas